<a href="https://colab.research.google.com/github/Aleksandr62aa/Test_task/blob/main/Test_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Importing libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image as im
from google.colab.patches import cv2_imshow

In [49]:
# Configuration parameters

# Rotation angle big
angle_big = [0, 5, 10, 15, 20, 25]  # 20

# Size small
k_small = [1, 1.5, 2] # 1.5

# Detected region big
x_big = [0, 1000, 2000, 3000, 4000, 5000, 6000] # 3000
y_big = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000] # 3000

# Coordinates of region big
dx_big = 2500
dy_big = 4500

# Saving results
results = {}

# Background
background = 0

In [50]:
# Conv image PIL -> numpy
def conv_np(img_pil):
    return np.array(img_pil.convert('RGB'))[:, :, ::-1].copy()

# Rotation image PIL
def rot_angle(img_pil, angle, background):
    return img_pil.rotate(angle, fillcolor=(background, background, background), expand=True)

# Template matching methods
def match_template(image, template):
    # template matching
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    #  coordinates of the detected region
    x1, y1 = max_loc
    return x1, y1, max_val

In [51]:
# Loading image big
big_pil = Image.open('big.tif')

if big_pil is None:
    raise FileNotFoundError("images big.tif not found")

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (106587012 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [52]:
# Conv big_pil -> numpy
big_np = conv_np(big_pil)

# Size small
big_height, big_width = big_np.shape[:2]

big_height, big_width

(12698, 8394)

In [53]:
# Loading image small
small_pil = Image.open('small.png')

if small_pil is None:
    raise FileNotFoundError("images small.png not found")

In [54]:
# Conv small_pil -> numpy
small_np = conv_np(small_pil)

# Size small
small_height, small_width = small_np.shape[:2]

small_height, small_width

(960, 540)

In [55]:
#  Process

for angle in angle_big:
    # Rotation angle big
    big_angle = rot_angle(big_pil, angle, background)
    # Conv -> numpy big
    big_angle_np = conv_np(big_angle)
    # Converting images to grayscale
    big_gray = cv2.cvtColor(big_angle_np, cv2.COLOR_BGR2GRAY)

    for k in k_small:
        # Resizing images small
        width = int(small_width * k)
        height = int(small_height * k)
        small_np_k = cv2.resize(small_np, (width, height))
        # Converting images to grayscale
        small_gray = cv2.cvtColor(small_np_k , cv2.COLOR_BGR2GRAY)

        for x in x_big:
            for y in y_big:

                # Apply template matching
                result = match_template(big_gray[y:y+dy_big, x:x+dx_big],
                                        small_gray)

                results[result[2]] = {'x1': result[0] + x,
                                      'y1': result[1] + y,
                                      'x2': result[0] + x + width,
                                      'y2': result[1] + y + height,
                                      'angle': angle,
                                      }

In [56]:
# Get the coordinates of the detected region
result = results[max(results.keys())]
x1, y1, x2, y2, angle =  [*result.values()]
x1, y1, x2, y2, angle, max(results.keys())

(3466, 5729, 4276, 7169, 20, 0.30329543352127075)

In [57]:
# Draw a rectangle around the detected region

# Rotation angle big
big_angle = rot_angle(big_pil, angle, background)

# Conv -> numpy big
big_angle_np = conv_np(big_angle)

# Draw a rectangle around the detected region
cv2.rectangle(big_angle_np, (x1, y1), (x2, y2), (0, 0, 0), 10)

# Save the results
cv2.imwrite('big_template.jpg', big_angle_np[:,:,:])

True

In [58]:
# Conv numpy big -> PIL big
big_angle_pil = im.fromarray(big_angle_np[:,:,:])

# Rotation angle big
big_pil_template = rot_angle(big_angle_pil, -angle, background)

# Save the results
big_pil_template.save('big_template_0.jpg')